In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.image as mpimg 
import pathlib
import numpy as np

C:\Users\kanan\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kanan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\kanan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
X_dir="images/images/"
y_dir="annotations/annotations"

In [3]:

def train_dataframe(which="trainval"):
    df=pd.read_csv(y_dir+"/"+which+".txt",sep=" ",names=["File_name","ID","Species","Breed_ID"])
    file_path=X_dir+df["File_name"]+".jpg"
    file_path=tf.constant(file_path)
    target_path=y_dir+"/trimaps/"+df["File_name"]+".png"
    target_path=tf.constant(target_path)
    return tf.data.Dataset.from_tensor_slices((file_path,target_path))

In [4]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [5]:
@tf.function
def read_image(input_path,mask_path): # read image from file
    X=tf.io.read_file(input_path)
    X=tf.image.decode_jpeg(X,channels=3)
    X=tf.image.resize(X,(572,572))
    y=tf.io.read_file(mask_path)
    y=tf.image.decode_png(y,channels=3)
    y=tf.image.resize(y,(388,388))
    return X,y

@tf.function
def pre_process(input_image,input_mask):
    input_image=input_image/255.0
    input_mask -=1
    return input_image,input_mask    

In [6]:
df_train=train_dataframe()
df_train=df_train.map(read_image)
df_train=df_train.map(pre_process)
df_test=train_dataframe("test")

In [7]:
train_datasets=df_train.shuffle(1000).repeat(5).batch(5)
test_datasets=df_test.batch(100)

In [8]:
train_datasets

<BatchDataset shapes: ((None, 572, 572, 3), (None, 388, 388, 3)), types: (tf.float32, tf.float32)>

In [9]:
class Double_Conv_Down(keras.layers.Layer):
    
    def __init__(self,n_layers,in_c,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Conv2D(in_c,kernel_size=3) for i in range(n_layers)]
        self.activation=keras.layers.Activation(activation)
        self.max_pool=keras.layers.MaxPool2D(pool_size=2,strides=2)
        
    def call(self,inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z)
            Z=self.activation(Z)
        self.next_use=Z
        Z=self.max_pool(Z)
        return Z
        
        

In [10]:
class Double_Conv(keras.layers.Layer):
    
    def __init__(self,n_layers,in_c,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Conv2D(in_c,kernel_size=3) for i in range(n_layers)]
        self.activation=keras.layers.Activation(activation)
        
    def call(self,inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z)
            Z=self.activation(Z)
        return Z

In [11]:
class UNet(keras.Model):
    
    def __init__(self,n_times,u_times,**kwargs):
        super().__init__(**kwargs)
        Z=[64*2**i for i in range(n_times)]
        print(Z)
        self.my_conv=[Double_Conv_Down(2,Z[i]) for i in range(n_times)]
        self.double_conv=[keras.layers.Conv2D(Z[n_times-1]*2,kernel_size=3) for i in range(u_times)]
        self.transpose1=keras.layers.Conv2DTranspose(512,kernel_size=2,strides=(2,2))
        self.up_conv1=Double_Conv(2,512)
        self.transpose2=keras.layers.Conv2DTranspose(256,kernel_size=2,strides=(2,2))
        self.up_conv2=Double_Conv(2,256)
        self.transpose3=keras.layers.Conv2DTranspose(128,kernel_size=2,strides=(2,2))
        self.up_conv3=Double_Conv(2,128)
        self.transpose4=keras.layers.Conv2DTranspose(64,kernel_size=2,strides=(2,2))
        self.up_conv4=Double_Conv(2,64)
        self.out=keras.layers.Conv2D(3,kernel_size=1)
        
    def resize(self,input_tensor,output_tensor):
        input_size=input_tensor.shape[2]
        output_size=output_tensor.shape[2]
        delta=(input_size-output_size)//2
        return input_tensor[:,delta:input_size-delta,delta:input_size-delta,:]
        
    def call(self,inputs):
        Z=inputs
        for layer1 in self.my_conv:
            Z=layer1(Z)
        for layers in self.double_conv:
            Z=layers(Z)
            
        Z=self.transpose1(Z)  
        Y=self.resize(self.my_conv[3].next_use,Z)
        Z=tf.concat([Z,Y],0)
        Z=self.up_conv1(Z)
        
        Z=self.transpose2(Z)  
        Y=self.resize(self.my_conv[2].next_use,Z)
        Z=tf.concat([Z,Y],0)
        Z=self.up_conv2(Z)
        
        Z=self.transpose3(Z)  
        Y=self.resize(self.my_conv[1].next_use,Z)
        Z=tf.concat([Z,Y],0)
        Z=self.up_conv3(Z)
        
        Z=self.transpose4(Z)  
        Y=self.resize(self.my_conv[0].next_use,Z)
        Z=tf.concat([Z,Y],0)
        Z=self.up_conv4(Z)
        
        Z=self.out(Z)
        
        return Z
        

In [12]:
model=UNet(4,2)
optimizer=keras.optimizers.Adam(lr=0.001)
model.compile(loss="mse",optimizer=optimizer)

[64, 128, 256, 512]


In [ ]:
model.fit(train_datasets,epochs=5,batch_size=5)

Epoch 1/5


In [13]:
iterator = tf.compat.v1.data.make_one_shot_iterator(train_datasets)
data_X, data_y = iterator.get_next()

In [16]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.